# Python: Unit Testing and Debugging

## <font color='orange'>Required Reading</font> 
1. [Debugging](http://swcarpentry.github.io/python-novice-inflammation/09-debugging/index.html) from Software Carpentry. This resource will guide you through 
2. [Understanding Unit Testing](https://jeffknupp.com/blog/2013/12/09/improve-your-python-understanding-unit-testing/) by Jeff Knupp. This tutorial guides the reader through the basics of unit testing and using Python's [unittest](https://docs.python.org/3/library/unittest.html#assert-methods) package.

## Optional Reading
Components that come up frequently with debugging and unit testing are errors and exceptions. To learn more about how they work in Python, read this tutorial on [Errors and Exceptions](http://swcarpentry.github.io/python-novice-inflammation/07-errors/index.html) from Software Carpentry.

# Practical Example
This may come as a surprise, but as a fellow human, we all make mistakes. As explained in the above resources, we can either fix code when it breaks (a reactive approach) or take steps to ensure we write good code at each step along the way anticipating the needs of each component of our program through unit testing (a more proactive approach).

For a working example here, let's start with our `ismultiple()` function from earlier in the tutorial. As a reminder, this function took two arguments: (1) a number and (2) a possible multiple of the number. Here's the function we had for that:

In [12]:
# Determine if one number is a multiple of another
def ismultiple(number,multiple):
    if number % multiple == 0:
        return True
    else:
        return False

In an ideal world, we'd create our unit tests BEFORE we wrote the code, that way we could check along the way to know when we'd satisfied all of the conditions we would need to meet. So let's first list our requirements that we'll want to test for this code. Of course, there are the requirements that you or your colleagues need to decide on for what "correct" functionality should be. These will be specified by the needs of the project. I want to be extremely thorough for this example (probably overly so for this example), so you'll see there are a surprising number when you try to cover everything. 

1. The function should return the correct answer for known examples including:
    1. 2 is a multiple of 6
    2. 3 is a multiple of 12
    3. 4 is not a multiple of 9
    4. 13 is not a multiple of 7
2. 1 is a multiple of all numbers greater than or equal to 1
3. A number is always a multiple of itself
4. A number has no multiples larger than itself
5. Negative numbers and zero are invalid inputs and if a negative number is input, it should return False
6. Any non-integer arguments should cause the function to return false

In [14]:
import unittest

class IsmultipleTest(unittest.TestCase):
    """Tests for `ismultiple()`."""

    def test_known_values(self):
        """1) It should return the correct answer for known examples"""
        self.assertEqual(ismultiple(6,  2 ), True)
        self.assertEqual(ismultiple(12, 3 ), True)
        self.assertEqual(ismultiple(9,  4 ), False)
        self.assertEqual(ismultiple(7,  13), False)
        
    def test_1(self):
        """2) 1 is a multiple of all numbers greater than or equal to 1"""
        self.assertEqual(ismultiple(1 ,1), True)
        self.assertEqual(ismultiple(2 ,1), True)
        self.assertEqual(ismultiple(3 ,1), True)
        self.assertEqual(ismultiple(99,1), True)
        
    def test_all_multiples_of_themselves(self):
        """3) A number is always a multiple of itself"""
        self.assertEqual(ismultiple( 2, 2), True)
        self.assertEqual(ismultiple( 3, 3), True)
        self.assertEqual(ismultiple(99,99), True)
        
    def test_no_multiples_larger_than_itself(self):
        """4) A number has no multiples larger than itself"""
        self.assertEqual(ismultiple(2,99), False)
        self.assertEqual(ismultiple(3, 4), False)
        self.assertEqual(ismultiple(4, 5), False)
        
    def test_negative_and_zero(self):
        """5) Negative numbers and zero are invalid inputs and if a
        negative number is input, it should return False"""
        self.assertEqual(ismultiple(-50,  10), False)
        self.assertEqual(ismultiple( 50, -10), False)
        self.assertEqual(ismultiple(-50, -10), False)
        self.assertEqual(ismultiple(  0,  10), False)
        self.assertEqual(ismultiple( 50,   0), False)
        self.assertEqual(ismultiple(  0,   0), False)
        
    def test_non_integer(self):
        """6) Any non-integer arguments should cause the function to return false"""
        self.assertEqual(ismultiple('string',3), False)
        self.assertEqual(ismultiple(3,'string'), False)
        self.assertEqual(ismultiple('string','string'), False)
        self.assertEqual(ismultiple(7.5, 4), False)
        self.assertEqual(ismultiple(8, 4.1), False)
        self.assertEqual(ismultiple([6,4],2), False)
        self.assertEqual(ismultiple(12,(1,4)), False)
        
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)
    # If this was in a file instead of a Jupyter Notebook, you would right 
    #   the following instead of the line above:
    # unittest.main()

...F.E
ERROR: test_non_integer (__main__.IsmultipleTest)
6) Any non-integer arguments should cause the function to return false
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-14-5b4049620fa2>", line 44, in test_non_integer
    self.assertEqual(ismultiple('string',3), False)
  File "<ipython-input-12-0fdbf0b1e810>", line 3, in ismultiple
    if number % multiple == 0:
TypeError: not all arguments converted during string formatting

FAIL: test_negative_and_zero (__main__.IsmultipleTest)
5) Negative numbers and zero are invalid inputs and if a
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-14-5b4049620fa2>", line 35, in test_negative_and_zero
    self.assertEqual(ismultiple(-50,  10), False)
AssertionError: True != False

----------------------------------------------------------------------
Ran 6 tests in 0.012s

FAILED (failu

These tests have 3 possible outcomes:
1. OK, The test passes. This is represented with a '`.`' at the top of the output.
2. FAIL: the test does not pass, and raises an AssertionError exception.
3. ERROR: The test raises an exception other than AssertionError.

In this case the outcome is:
```
...F.E
```
so there of the 6 tests presented, two situations were not adequate:
- FAILED: 5) Negative numbers and zero are invalid inputs and if a negative number is input, it should return False
- ERROR: 6) Any non-integer arguments should cause the function to return false



Let's try to address the issues above. To do this, any invalid input should cause the function to return `False`. This includes non-integer arguments, zero values, and negative numbers.

Below you'll see the `isinstance(a,b)` function, which checks to see whether `a` is an instance of (or is a type of) `b`. So if `a` is an integer, and `b` is `int`, then this would return `True`.

In [7]:
# Determine if one number is a multiple of another
def ismultiple(number,multiple):
    
    # Check the input to see if it's valid
    if isinstance(number,int) and isinstance(multiple,int):
        if number <= 0 or multiple <= 0:
            return False
    else:
        return False
    
    # Check if one numbers ia multiple of another
    if number % multiple == 0:
        return True
    else:
        return False

In [10]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

......
----------------------------------------------------------------------
Ran 6 tests in 0.007s

OK


Excellent! Now we've passed all the tests and we can be confident that our function meets our particularly stringent specifications. If this was a function that we were giving to other users, we may very well want to be sure that our tests represent the desired behavior so those users could benefit from a well-functioning program. We also know that we can build on this function in a larger code base with confidence in its operation.